In [1]:
!pip install -q langchain==0.0.219 openai==0.27.8

# !pip install -q transformers==4.29.2 sentencepiece==0.1.99 accelerate==0.19.0 bitsandbytes==0.39.0

!pip install -q python-dotenv==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00


In [6]:
from langchain.llms import OpenAI
from dotenv import dotenv_values
load_dotenv =  dotenv_values("./app.env")

openai_api_key = load_dotenv["OPEN_API_KEY"]


In [7]:
llm = OpenAI(model_name="gpt-3.5-turbo",openai_api_key=openai_api_key, temperature=0.9)

# Listing parsing

In [8]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts.prompt import PromptTemplate

output_parser = CommaSeparatedListOutputParser()
fromat_instructions = output_parser.get_format_instructions()
print(fromat_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [14]:
prompt = PromptTemplate(input_variables=["typee"],
                        template= "I want to know ingradient of {typee} pasta.\n{fromat_instructions}",
                        partial_variables={"fromat_instructions":fromat_instructions})

print(prompt.format(typee="Egyptian"))




I want to know ingradient of Egyptian pasta.
Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [15]:
pro_prompt =  prompt.format(typee="Egyptian")

response = llm(pro_prompt)
print(response)

output = output_parser.parse(response)
print(output)

flour, water, salt
['flour', 'water', 'salt']


# DateTime Parsing

In [16]:
format_instructions = "answer the question with datetime format like  03/05/2020 and location like London/England"

prompt =  PromptTemplate(template = "When {name} who is famour born  and where ? \n {format_instructions}",
                         input_variables =["name"],
                         partial_variables={"format_instructions":format_instructions})

print(prompt.format(name="albert einstein"))



When albert einstein who is famour born  and where ? 
 answer the question with datetime format like  03/05/2020 and location like London/England


In [18]:
pro_prompt =  prompt.format(name="albert einstein")

response = llm(pro_prompt)
print(response)


03/14/1879 in Ulm, Germany


In [19]:
pro_prompt =  prompt.format(name="Mahmoud Reda")

response = llm(pro_prompt)
print(response)


09/01/1930 in Cairo, Egypt


# PyDantic Parsing

it is a format to return the parsing as class format so we can use it in other applications like fastApi for example

In [20]:
from typing import List
from pydantic import BaseModel , Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts.prompt import PromptTemplate


In [21]:
class pydantci(BaseModel):
  platename : str = Field(default_factory=str)
  ingradient : List[str] = Field(default_factory=list)

In [22]:
parser  = PydanticOutputParser(pydantic_object=pydantci)

format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"platename": {"title": "Platename", "type": "string"}, "ingradient": {"title": "Ingradient", "type": "array", "items": {"type": "string"}}}}
```


In [23]:
prompt = PromptTemplate(input_variables=["typee"],
                        template= "I want to know ingradient of {typee} pasta.\n{fromat_instructions}",
                        partial_variables={"fromat_instructions":format_instructions})


print(prompt.format(typee="Egyptian"))

I want to know ingradient of Egyptian pasta.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"platename": {"title": "Platename", "type": "string"}, "ingradient": {"title": "Ingradient", "type": "array", "items": {"type": "string"}}}}
```


In [24]:
pro = prompt.format(typee="Egyptian")

response = llm(pro)
print(response)

{
  "platename": "Egyptian Pasta",
  "ingradient": [
    "Pasta",
    "Garlic",
    "Onion",
    "Tomato paste",
    "Spices (such as cumin, coriander, paprika)",
    "Water",
    "Vegetable oil"
  ]
}


In [25]:
output =  parser.parse(response)
print(output)

platename='Egyptian Pasta' ingradient=['Pasta', 'Garlic', 'Onion', 'Tomato paste', 'Spices (such as cumin, coriander, paprika)', 'Water', 'Vegetable oil']


In [26]:
type(output)

__main__.pydantci